# DSCI100 Project Final Report
### Players Subscription Predictive Data Analysis

### Group 29:
- Ysabel Maria
- Sanjana Gopee - 59940676
- Simar Pandher - 14521397
- Olivia Kong

## Instructions for Final Report
- Title **(Done)**

- Introduction: Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report
clearly state the question you tried to answer with your project identify and fully describe the dataset that was used to answer the question **(Done)**

- Methods & Results: Describe the methods you used to perform your analysis from beginning to end that narrates the analysis code. **(In-progress)**

Your report should include code which:

1. Loads data 
2. Wrangles and cleans the data to the format necessary for the planned analysis
3. Performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
4. Creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
5. Performs the data analysis
6. Creates a visualization of the analysis 

Note: All figures should have a figure number and a legend

- Discussion:
1. Summarize what you found
2. Discuss whether this is what you expected to find?
3. Discuss what impact could such findings have?
4. Discuss what future questions could this lead to?

- References
You may include references if necessary, as long as they all have a consistent citation style.

In [1]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate



alt.data_transformers.enable('vegafusion')

set_config(transform_output="pandas")

### Introduction
This project is based on datasets that have been provided by a research group in Computer Science at UBC who obtained data regarding how individuals play video games. They did this through recording player actions in a MineCraft Server, and collecting data regarding information about each individual as well as how they play. This data has been condensed into the datasets of **Players** and **Sessions**. The question our group has chosen to answer is Question 3 provided in the criteria. 

**Question 3**: We would like to know something about our populations of users, in particular, we would like to have a good model of whether or not a player will continue contributing given past participation. 

## Data Description

## Players Data
  The players.csv filed is a data set containing information about the players in the game. There are 196 observations with data about the players such as their experience, whether they subscribe, their email, the number of hours played, their names, gender and age. These categories are split into the 9 variables (column names) below. 

  


**Players.csv:**

|     Variable     |  Type   |                    Description                            |
|------------------|---------|-----------------------------------------------------------|
| experience       | String  | The level of expertise that the player has in the game. Possible values are "Amateur", "Beginner", "Regular", "Pro", "Veteran". This is a categorical variable.              |
| subscribe        | Boolean | Whether or not the player has subscribed to the game. The variable can only take the value "TRUE" or "FALSE", indicating "yes"  or "no" to whether they are subscribed.  |
| hashedEmail      | String  | This is a string of letters and numbers to encrypt the email of the user. This is a unique identifier for each player.                        |
| played_hours     | Float   | The played hours indicates the number of hours spent playing the game approximated to one decimal place.                               |
| name             | String  | This is the name (first name) of the player. This is probably not a unique identifier since two people could coincidentally have the same name.                                             |
| gender           | String  | Gender is a categorical variable which has the following possible values: "Male", "Female", "Non-binary", "Prefer not to say", "Agender", "Two-spirited", "Other".                                        |
| age              | Integer | Player's age                                              |
| individualId     | N/A     | Individual ID of the player, values were not provided therefore the category is essentially useless in data analysis           |
| organizationName | N/A     | Organization name, values were not provided making this category useless just as the "IndividualID" column is.                  |

The final two columns are negligible, therefore will be dropped and the categories of use are reduced to 7 variables from 9.

## Sessions Data
The sessions.csv data set contains specific data about the playing sessions of the players in the game. There are 1535 observations in the sessions data set. It has variables like the players' email, start time, end time, original start time and original end time.

**Sessions.csv:**
|     Variable     |  Type   |                    Description                            |
|------------------|---------|-----------------------------------------------------------|
| hashedEmail      | String  | This is a string of letters and numbers to encrypt the email of the user. This is a unique identifier for each player.                       |
| start_time     | String   | This includes the date - in format DD/MM/YY - and the time the player started playing the game. The time is in 24-hour format.                               |
| end_time             | String  | This includes the date - in format DD/MM/YY - and the time the player stopped playing the game. The time is in 24-hour format.                                             |
| original_start_time           | Integer  | This variable is a 14-digit integer indicating the start time.                                           |
| original_end_time                | Integer | This variable is a 14-digit integer indicating the end time.                                              |

Unlike the players data, none of the variables are negligible as they all provide information about the playing sessions rather than being empty observations.


## Method

### Summary
Using data available about **Players** and **Sessions**, the goal of this project is to predict player retention, whether a player will continue playing the game based on the data from the two given dataframes. The column "subscribe" is the categorical variable we are trying to predict, and this is based on our assumption that a players subscribing to the game will continue to play it. The two varaibles chosen to predict this are:

1. Total number of hours played
2. Average session time per player

This will require a KNN classification model with the most optimal K value, and the two above quantitative predictor variables. The data will also be split into training and test sets to evaluate the classifer model's performance using accuracy, recall and precision. Through cross validation, an optimal k-value will be chosen. 

### Detailed Steps [INCOMPLETE - Still need to write more]
- **Data Wrangling**:

1. Converting dtype object to numeric
2. Groupby() and mean
3. Merging the two data sets based on hashedemail

- **Exploratory Analysis**: explain
- **Data Analysis**: explain
- **Visualization of Analysis**: explain

### Data Analysis
#### Players Data

In [7]:
# Load the dataset
players = pd.read_csv("players.csv")
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [21]:
# Drop the columns "individualID" and "organizationName" as they provide no observations, therefore are useless
players_tidy = players.drop(columns=['individualId', 'organizationName'])
players_tidy

,experience,subscribe,hashedEmail,played_hours,name,gender,age
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21
...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17


#### Sessions Data

In [9]:
# Load the Dataset
sessions = pd.read_csv("sessions.csv")
sessions

,hashedEmail,start_time,end_time,original_start_time,original_end_time
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,30/06/2024 18:12,30/06/2024 18:24,1.719770e+12,1.719770e+12
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,17/06/2024 23:33,17/06/2024 23:46,1.718670e+12,1.718670e+12
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,25/07/2024 17:34,25/07/2024 17:57,1.721930e+12,1.721930e+12
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,25/07/2024 03:22,25/07/2024 03:58,1.721880e+12,1.721880e+12
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,25/05/2024 16:01,25/05/2024 16:12,1.716650e+12,1.716650e+12
...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,10/05/2024 23:01,10/05/2024 23:07,1.715380e+12,1.715380e+12
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,01/07/2024 04:08,01/07/2024 04:19,1.719810e+12,1.719810e+12
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,28/07/2024 15:36,28/07/2024 15:57,1.722180e+12,1.722180e+12
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,25/07/2024 06:15,25/07/2024 06:22,1.721890e+12,1.721890e+12


In [11]:
# Convert all columns except for "hashedEmail" into datetime rather than str
sessions['start_time_final']=pd.to_datetime(sessions["start_time"], dayfirst=True)
sessions['end_time_final']=pd.to_datetime(sessions["end_time"], dayfirst=True)
sessions

,hashedEmail,start_time,end_time,original_start_time,original_end_time,start_time_final,end_time_final
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,30/06/2024 18:12,30/06/2024 18:24,1.719770e+12,1.719770e+12,2024-06-30 18:12:00,2024-06-30 18:24:00
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,17/06/2024 23:33,17/06/2024 23:46,1.718670e+12,1.718670e+12,2024-06-17 23:33:00,2024-06-17 23:46:00
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,25/07/2024 17:34,25/07/2024 17:57,1.721930e+12,1.721930e+12,2024-07-25 17:34:00,2024-07-25 17:57:00
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,25/07/2024 03:22,25/07/2024 03:58,1.721880e+12,1.721880e+12,2024-07-25 03:22:00,2024-07-25 03:58:00
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,25/05/2024 16:01,25/05/2024 16:12,1.716650e+12,1.716650e+12,2024-05-25 16:01:00,2024-05-25 16:12:00
...,...,...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,10/05/2024 23:01,10/05/2024 23:07,1.715380e+12,1.715380e+12,2024-05-10 23:01:00,2024-05-10 23:07:00
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,01/07/2024 04:08,01/07/2024 04:19,1.719810e+12,1.719810e+12,2024-07-01 04:08:00,2024-07-01 04:19:00
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,28/07/2024 15:36,28/07/2024 15:57,1.722180e+12,1.722180e+12,2024-07-28 15:36:00,2024-07-28 15:57:00
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,25/07/2024 06:15,25/07/2024 06:22,1.721890e+12,1.721890e+12,2024-07-25 06:15:00,2024-07-25 06:22:00


In [14]:
# Calculate Session Length in days and time, adding a "session_length" column
sessions["session_length"] = sessions["end_time_final"]-sessions["start_time_final"]
sessions

,hashedEmail,start_time,end_time,original_start_time,original_end_time,start_time_final,end_time_final,session_length
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,30/06/2024 18:12,30/06/2024 18:24,1.719770e+12,1.719770e+12,2024-06-30 18:12:00,2024-06-30 18:24:00,0 days 00:12:00
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,17/06/2024 23:33,17/06/2024 23:46,1.718670e+12,1.718670e+12,2024-06-17 23:33:00,2024-06-17 23:46:00,0 days 00:13:00
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,25/07/2024 17:34,25/07/2024 17:57,1.721930e+12,1.721930e+12,2024-07-25 17:34:00,2024-07-25 17:57:00,0 days 00:23:00
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,25/07/2024 03:22,25/07/2024 03:58,1.721880e+12,1.721880e+12,2024-07-25 03:22:00,2024-07-25 03:58:00,0 days 00:36:00
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,25/05/2024 16:01,25/05/2024 16:12,1.716650e+12,1.716650e+12,2024-05-25 16:01:00,2024-05-25 16:12:00,0 days 00:11:00
...,...,...,...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,10/05/2024 23:01,10/05/2024 23:07,1.715380e+12,1.715380e+12,2024-05-10 23:01:00,2024-05-10 23:07:00,0 days 00:06:00
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,01/07/2024 04:08,01/07/2024 04:19,1.719810e+12,1.719810e+12,2024-07-01 04:08:00,2024-07-01 04:19:00,0 days 00:11:00
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,28/07/2024 15:36,28/07/2024 15:57,1.722180e+12,1.722180e+12,2024-07-28 15:36:00,2024-07-28 15:57:00,0 days 00:21:00
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,25/07/2024 06:15,25/07/2024 06:22,1.721890e+12,1.721890e+12,2024-07-25 06:15:00,2024-07-25 06:22:00,0 days 00:07:00


In [17]:
#Grouping and Calculating Mean in Minutes
sessions_group=sessions.groupby('hashedEmail')["session_length"].mean().reset_index()
sessions_group["session_length_mins"] = sessions_group["session_length"].dt.total_seconds()/60
sessions_group

,hashedEmail,session_length,session_length_mins
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,0 days 00:53:00,53.000000
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,0 days 00:30:00,30.000000
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,0 days 00:11:00,11.000000
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,0 days 00:32:09.230769230,32.153846
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,0 days 00:35:00,35.000000
...,...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,0 days 00:16:00,16.000000
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,0 days 01:20:00,80.000000
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,0 days 00:15:28.064516129,15.467742
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,0 days 00:09:00,9.000000


#### Merging Players Data and Sessions Data

In [ ]:
# Merging datasets based on hashedEmail and dropping irrelevant columns (we are only intersted in hours played and whether or not each individual is subscribed)
merged=sessions_group.merge(players, on='hashedEmail')
merged.drop(columns=["session_length", "individualId", "organizationName", "name", "gender","age","experience"])

#### Solving Data Imbalance

In [49]:
# Use value_counts() to view the ratio of subscribed individuals to not subscribed individuals
merged["subscribe"].value_counts()

subscribe
True     93
False    32
Name: count, dtype: int64

This shows that there are way more people who are subscribed than not subscribed, which will cause issues in later classification. Therefore, we want to make the data more even by replicating the less common result (False) multiple times. 

In [66]:
subscribe_false = merged[merged["subscribe"] == False]
subscribe_true = merged[merged["subscribe"] == True]

subscribe_false_upsample = subscribe_false.sample(
n=subscribe_true.shape[0], replace=True)

upsampled_merged = pd.concat((subscribe_false_upsample, subscribe_true))
(upsampled_merged["subscribe"].value_counts())

subscribe
False    93
True     93
Name: count, dtype: int64

This output tells us that the values are now even after the less common result (False) has been replicated up to the same number of values for the more common result (True).